In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from src.config import DATA_PROCESSED
train_df = pd.read_csv(DATA_PROCESSED / "train_data.csv")
test_df = pd.read_csv(DATA_PROCESSED / "test_data.csv")
FEATURES = [
    "day_of_week",
    "is_weekend",
    "day_of_year",
    "qty_lag_1",
    "qty_lag_7",
    "qty_lag_14",
    "qty_roll_7",
    "qty_roll_14",
    "is_holiday",
    "days_to_holiday",
    "days_since_holiday",
    "is_fasting"
]

TARGET = "target_qty"

X_train = train_df[FEATURES]
y_train = train_df[TARGET]

X_test = test_df[FEATURES]
y_test = test_df[TARGET]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

# SKU Level MAE to know which product is predictable
test_df["prediction"] = y_pred
test_df["abs_error"] = abs(test_df["prediction"] - test_df["target_qty"])

sku_mae = (
    test_df.groupby("sku_id")["abs_error"]
    .mean()
    .sort_values(ascending=False)
)

print(sku_mae.head(513))


MAE: 3.2276853413185433
sku_id
TOP-DRI004    11.289736
SUN-BAL001    11.285942
ድንብ-FRU001    10.141077
BAN-FRU001    10.127757
POT-FRU001    10.102498
                ...    
VVG-CAN003     1.654136
TES-CAN002     1.649862
DAC-CAN002     1.648418
FAR-HOU002     1.646366
TUM-HOU009     1.639734
Name: abs_error, Length: 513, dtype: float64
